In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re

visited_urls = set()
urls_to_crawl = ['https://dantri.com.vn/']
start_domain = urlparse(urls_to_crawl[0]).netloc
articles_collected_count = 0
MAX_ARTICLES = 5

def is_valid_url(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def is_article_page(url, soup):
    if '.htm' in url.split('/')[-1] and urlparse(url).path.count('/') > 1:
        content_tag = soup.find('div', class_='article-content')
        if content_tag:
            return True
    return False

def crawlerThread():
    global articles_collected_count

    while urls_to_crawl and articles_collected_count < MAX_ARTICLES:
        current_url = urls_to_crawl.pop(0)

        if current_url in visited_urls:
            continue

        print(f"Đang thu thập: {current_url}")
        visited_urls.add(current_url)

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
            }
            response = requests.get(current_url, headers=headers, timeout=5)

            if response.status_code == 200 and 'text/html' in response.headers.get('Content-Type', ''):
                soup = BeautifulSoup(response.text, 'html.parser')

                if is_article_page(current_url, soup):
                    if articles_collected_count < MAX_ARTICLES:
                        content_tag = soup.find('div', class_='article-content')
                        content = content_tag.get_text(strip=True) if content_tag else "Không có nội dung"

                        print("\n========== BÀI BÁO ĐÃ THU THẬP ==========")
                        print(f"URL: {current_url}")
                        print(f"Nội dung (200 ký tự đầu): {content[:200]}...")
                        print("=========================================\n")

                        articles_collected_count += 1

                if articles_collected_count < MAX_ARTICLES:
                    for link_tag in soup.find_all('a', href=True):
                        href = link_tag.get('href')
                        full_url = urljoin(current_url, href)

                        if is_valid_url(full_url) and urlparse(full_url).netloc == start_domain:
                            if '.htm' in full_url.split('/')[-1] and full_url not in visited_urls and full_url not in urls_to_crawl:
                                urls_to_crawl.append(full_url)

        except Exception as e:
            print(f"Lỗi khi thu thập {current_url}: {e}")

        time.sleep(1)

if __name__ == "__main__":
    crawlerThread()
    print(f"Collect: {articles_collected_count} Paper.")
    print(f"Connect: {len(visited_urls)} URL.")

Đang thu thập: https://dantri.com.vn/
Đang thu thập: https://dantri.com.vn/thoi-su/dai-hoi-dang.htm
Đang thu thập: https://dantri.com.vn/event/trong-cay-xanh-tren-duong-cao-toc-nen-hay-khong-7314.htm

========== BÀI BÁO ĐÃ THU THẬP ==========
URL: https://dantri.com.vn/event/trong-cay-xanh-tren-duong-cao-toc-nen-hay-khong-7314.htm
Nội dung (200 ký tự đầu): Thứ 5, 18/09/2025Cây xanh tươi tốt ở dải phân cách giữa cao tốc Pháp Vân - Cầu Giẽ...

Đang thu thập: https://dantri.com.vn/event/nghi-quyet-71-ve-dot-pha-phat-trien-giao-duc-7172.htm

========== BÀI BÁO ĐÃ THU THẬP ==========
URL: https://dantri.com.vn/event/nghi-quyet-71-ve-dot-pha-phat-trien-giao-duc-7172.htm
Nội dung (200 ký tự đầu): Thứ 6, 19/09/2025Ba yếu tố làm nên một bộ sách giáo khoa bền vững...

Đang thu thập: https://dantri.com.vn/event/aqua-warriors-van-don-2025-camel-cup-7121.htm

========== BÀI BÁO ĐÃ THU THẬP ==========
URL: https://dantri.com.vn/event/aqua-warriors-van-don-2025-camel-cup-7121.htm
Nội dung (200 ký tự 